# CS435 Term Project

## Analyze Crime Rates to Help Mitigate Crimes in Chicago

Team Members: Jimit Bhalavat, Logan Cuccia, Kyle Cummings, Mike Peyton

### Description

Chicago was once a pillar of prosperity and trade where companies housed their headquarters, where business partners formed coalitions, and where impactful business decisions were made. Today, it is still a major capital for trade, and some areas are very prosperous. Other sectors are rampant with crime, murder, theft, break-ins, and kidnappings. The crimes within some Chicago districts are so high that officers often don’t know whether they will return home that day to their families. Although there are countless factors contributing to the growing crime rate and law enforcement procedures, crimes are overwhelmingly complex and have to do with politics, state, and local government actions. The question arises how has crime changed over the years? Is it possible to predict where or when a crime will be committed? Which areas of the city have evolved over this time span?

The goal of this project is to tackle how we can use data from these crimes to potentially stop them from becoming so frequent. It will also provide data summarizations such as the location and frequency of the crimes, and the nature of the crimes which can ultimately help mitigate crimes and help law enforcement agencies pinpoint where these crimes are more likely to occur, and which districts should have a higher abundance of police officers. This information can not only save the lives of citizens but also police officers who risk their lives every day to better their communities. It also ensures that these criminals have a higher likelihood of being caught since so many crimes in these districts go unsolved. The dataset contains attributes such as date, coordinates of the incident, district, the nature of the crime, etc. We will attempt to use these attributes to generate summarizations that will assist in the fight against crime as well as assist in the city’s economic future. 

### Goal of the Project

In order to solve this problem and help aid law enforcement agencies in Chicago, we will aim to analyze this dataset and run a variety of different processing techniques and models in order to interpret results and answer the questions mentioned above. This dataset is large and contains various attributes, so it is complex to analyze through normal methods. In order to process this data effectively and efficiently, we will use MapReduce to process the data and provide numerical summarizations such as the accuracy rates of prediction through Neural Networks, Random Forests, and K Means clustering. Other numerical summarizations will include the proportions of arrests made by the crimes committed, whether the trends in crimes have increased over the years, and if yes, what crimes are more likely to be committed. 


In this project, we aim to analyze the relationship between different types of crimes and the location they happened. In our research, we also aim to analyze the statistics of a few specific crimes: theft, homicide, and sexual harassment, and whether these crimes have declined or increased over the years. We aim to use a few of the packages available in Python3 such as Pandas (Dataframe), Numpy (Math), Seaborn and Matplotlib (Data Visualization), and Sklearn (Algorithms). The framework we plan to use is Hadoop or Spark. 


### Data Description

The dataset comes from Kaggle and reflects the reported incidents of crime in the City of Chicago from 2001 to 2017. The set excludes murders where data exists for each victim. This dataset is withdrawn from the Chicago Police Department’s Citizen Law Enforcement Analysis and Reporting System. The entire dataset spans around 2GB. In order to span the 16 years of data, the entire dataset has been broken down into four different CSV files. The entries of the files have attributes such as a unique identifier, block where the incident occurred, description of the location where the incident occurred, whether the incident was domestic-related, police district where the incident occurred, latitude, and longitude of the incident, to name a few. To protect the privacy of crime victims, the exact addresses of the crimes are not shared, but the location data included will allow us to pinpoint the general location of the crime. We can use the description attributes to categorize what type of crime was committed, and also filter based on other attributes such as whether an arrest was made, or whether the case was domestic.

In [1]:
import numpy as np
import pandas as pd
import matplotlib
from datetime import datetime

### Data Processing

In [2]:
dt_string = "09/05/2015 01:30:00 PM"

# Considering date is in dd/mm/yyyy format
dt_object1 = datetime.strptime(dt_string, "%d/%m/%Y %H:%M:%S %p")
print("dt_object1 =", dt_object1)


dt_object1 = 2015-05-09 01:30:00


In [3]:
crimes_temp = pd.read_csv("../Crimes_-_2001_to_Present.csv")
crimes_temp.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,...,8.0,44.0,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"


In [4]:
crimes = crimes_temp.drop(['IUCR', 'FBI Code', 'X Coordinate','Y Coordinate', 'Updated On', 'Location', 'Location Description'], axis = 1)
crimes = crimes.dropna()
crimes.isnull().sum()
crimes.head()

,ID,Case Number,Date,Block,Primary Type,Description,Arrest,Domestic,Beat,District,Ward,Community Area,Year,Latitude,Longitude
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,924,9.0,12.0,61.0,2015,41.815117,-87.670000
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,THEFT,POCKET-PICKING,False,False,1511,15.0,29.0,25.0,2015,41.895080,-87.765400
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,NARCOTICS,POSS: HEROIN(BRN/TAN),True,False,1412,14.0,35.0,21.0,2015,41.937406,-87.716650
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,ASSAULT,SIMPLE,False,True,1522,15.0,28.0,25.0,2015,41.881903,-87.755121
5,10224742,HY411435,09/05/2015 10:55:00 AM,082XX S LOOMIS BLVD,BURGLARY,FORCIBLE ENTRY,False,False,614,6.0,21.0,71.0,2015,41.744379,-87.658431


In [5]:
crimes['Description'] = crimes['Description'].str.replace(',', '/')

### Map Reduce

### Neural Networks

### K-Means Clustering

### Random Forests

In [6]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.functions import format_number, when, col, array, udf, lit
import pandas as pd
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, IndexToString
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.sql.types import *
import pyspark.sql.functions as F
import matplotlib.pyplot as plt

In [7]:

spark = SparkSession.builder.appName("Project - Chicago crime").config("spark.some.config.option", "some-value").config("spark.driver.memory", "8g").config("spark.driver.cores", "3").getOrCreate()

In [8]:

crimes_schema = StructType([StructField("ID", StringType(), True),
                            StructField("Case Number", StringType(), True),
                            StructField("Date", StringType(), True ),
                            StructField("Block", StringType(), True),
                            StructField("IUCR", StringType(), True),
                            StructField("Primary Type", StringType(), True  ),
                            StructField("Description", StringType(), True ),
                            StructField("Location Description", StringType(), True ),
                            StructField("Arrest", BooleanType(), True),
                            StructField("Domestic", BooleanType(), True),
                            StructField("Beat", StringType(), True),
                            StructField("District", StringType(), True),
                            StructField("Ward", StringType(), True),
                            StructField("Community Area", StringType(), True),
                            StructField("FBI Code", StringType(), True ),
                            StructField("X Coordinate", DoubleType(), True),
                            StructField("Y Coordinate", DoubleType(), True ),
                            StructField("Year", IntegerType(), True),
                            StructField("Updated On", DateType(), True ),
                            StructField("Latitude", DoubleType(), True),
                            StructField("Longitude", DoubleType(), True),
                            StructField("Location", StringType(), True )
                            ])

In [9]:
dataset = spark.read.option("header", "True").option("inferSchema", "true").csv("../Crimes_-_2001_to_present.csv")

In [10]:
dataset.limit(5).toPandas().head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12,61,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29,25,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,...,8,44,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,None
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,35,21,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,28,25,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"


In [11]:
dataset = dataset.filter((dataset['Primary Type']!='STALKING')&
                             (dataset['Primary Type']!='OBSCENITY')&
                             (dataset['Primary Type']!='NON-CRIMINAL (SUBJECT SPECIFIED)')&
                             (dataset['Primary Type']!='GAMBLING')&
                             (dataset['Primary Type']!='NON - CRIMINAL')&
                             (dataset['Primary Type']!='LIQUOR LAW VIOLATION')&
                             (dataset['Primary Type']!='PUBLIC INDECENCY')&
                             (dataset['Primary Type']!='HUMAN TRAFFICKING')&
                             (dataset['Primary Type']!='INTIMIDATION')&
                             (dataset['Primary Type']!='CONCEALED CARRY LICENSE VIOLATION')&
                             (dataset['Primary Type']!='NON-CRIMINAL')&
                             (dataset['Primary Type']!='OTHER NARCOTIC VIOLATION'))

In [12]:

dataset = dataset.withColumn("Day", F.split(dataset.Date, " ")[0])
dataset = dataset.withColumn("Day", F.to_date(dataset.Day, "MM/dd/yyyy"))
dataset = dataset.withColumn("Month", F.month(dataset.Day))
dataset = dataset.withColumn("WeekDay", F.dayofweek(dataset.Day))
dataset = dataset.withColumn("Hour", F.split(F.split(dataset.Date," ")[1], ":")[0].cast("int"))
dataset = dataset.na.drop()
dataset.limit(5).toPandas().head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Day,Month,WeekDay,Hour
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,1875917,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)",2015-09-05,9,7,1
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,1904869,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)",2015-09-04,9,6,11
2,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,1920384,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)",2015-09-05,9,7,12
3,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,1900086,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)",2015-09-05,9,7,1
4,10224742,HY411435,09/05/2015 10:55:00 AM,082XX S LOOMIS BLVD,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,...,1850165,2015,02/10/2018 03:50:01 PM,41.744379,-87.658431,"(41.744378879, -87.658430635)",2015-09-05,9,7,10


In [13]:

dataset = dataset.withColumn("Primary Type", \
                               when((dataset["Primary Type"] == 'CRIM SEXUAL ASSAULT') | \
                                    (dataset["Primary Type"] == 'PROSTITUTION') ,'SEX OFFENSE').otherwise(dataset['Primary Type']))
dataset = dataset.withColumn("Primary Type", \
                               when((dataset["Primary Type"] == 'WEAPONS VIOLATION') | \
                                    (dataset["Primary Type"] == 'INTERFERENCE WITH PUBLIC OFFICER') ,'PUBLIC PEACE VIOLATION').otherwise(dataset['Primary Type']))
dataset = dataset.withColumn("Primary Type", \
                               when((dataset["Primary Type"] == 'ARSON') ,'CRIMINAL TRESPASS').otherwise(dataset['Primary Type']))

In [14]:
dataset = dataset.na.drop()
dataset = dataset.drop('ID')
dataset = dataset.drop('Case Number')
dataset = dataset.drop('Block')
dataset = dataset.drop('IUCR')
dataset = dataset.drop('FBI Code')
dataset = dataset.drop('Updated On')
dataset = dataset.drop('X Coordinate')
dataset = dataset.drop('Y Coordinate')
dataset = dataset.drop('Description')
dataset = dataset.drop('Location')
dataset = dataset.drop('Date')
dataset.limit(5).toPandas().head(5)

,Primary Type,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,Year,Latitude,Longitude,Day,Month,WeekDay,Hour
0,BATTERY,RESIDENCE,False,True,924,9,12,61,2015,41.815117,-87.670000,2015-09-05,9,7,1
1,THEFT,CTA BUS,False,False,1511,15,29,25,2015,41.895080,-87.765400,2015-09-04,9,6,11
2,NARCOTICS,SIDEWALK,True,False,1412,14,35,21,2015,41.937406,-87.716650,2015-09-05,9,7,12
3,ASSAULT,APARTMENT,False,True,1522,15,28,25,2015,41.881903,-87.755121,2015-09-05,9,7,1
4,BURGLARY,RESIDENCE,False,False,614,6,21,71,2015,41.744379,-87.658431,2015-09-05,9,7,10


In [15]:
vector_assembler = VectorAssembler(inputCols = ["Latitude", "Longitude"], outputCol="features")
df_temp = vector_assembler.transform(dataset)
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(df_temp)


labelIndexer = StringIndexer(inputCol="Primary Type", outputCol="indexedLabel").fit(df_temp)
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)


(trainingData, testData) = df_temp.randomSplit([0.7, 0.3])
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10,impurity='gini', maxBins=32)

pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf,labelConverter])

model = pipeline.fit(trainingData)

predictions = model.transform(testData)

evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
evaluator2 = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="f1")

accuracy = evaluator.evaluate(predictions)
f1 = evaluator2.evaluate(predictions)


print("accuracy = %g" % (accuracy))
print("f1 score = %g" % (f1))

predictions.groupBy("predictedLabel").count().show()

accuracy = 0.252627
f1 score = 0.165528
+--------------+------+
|predictedLabel| count|
+--------------+------+
|         THEFT|939606|
|       BATTERY|858442|
|     NARCOTICS|212308|
+--------------+------+



In [16]:
from pyspark.ml.feature import OneHotEncoder
locationIndexer = StringIndexer(inputCol="Location Description", outputCol="indexedLocation").fit(dataset).transform(dataset)
encoded_data = OneHotEncoder(inputCols=["indexedLocation"],outputCols=["encodedLocation"]).fit(locationIndexer).transform(locationIndexer)

In [17]:

vector_assembler = VectorAssembler(inputCols = ["Latitude", "Longitude", "Arrest", "Beat", "Ward", "Community Area", "WeekDay", "Hour", "encodedLocation"], outputCol="features")
df_temp = vector_assembler.transform(encoded_data)
featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(df_temp)

labelIndexer = StringIndexer(inputCol="Primary Type", outputCol="indexedLabel").fit(df_temp)
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",labels=labelIndexer.labels)

(trainingData, testData) = df_temp.randomSplit([0.7, 0.3])
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10,impurity='gini', maxBins=32)

pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf,labelConverter])

model = pipeline.fit(trainingData)

predictions = model.transform(testData)

evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
evaluator2 = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="f1")

accuracy = evaluator.evaluate(predictions)
f1 = evaluator2.evaluate(predictions)

print("accuracy = %g" % (accuracy))
print("f1 score = %g" % (f1))

predictions.groupBy("predictedLabel").count().show()

accuracy = 0.329346
f1 score = 0.212419
+--------------+-------+
|predictedLabel|  count|
+--------------+-------+
|         THEFT|1162882|
|       BATTERY| 321095|
|     NARCOTICS| 495067|
|      BURGLARY|  31357|
+--------------+-------+



In [18]:
labels = labelIndexer.labels
prediction_probility= predictions.select("Primary Type","Latitude","Longitude","District", "Ward","probability")

def addCrimeIndex(li):
    dic = {}
    for i in range(len(labels)):
        dic[labels[i]]=li[i]
    return dic

probability_to_array = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))
prediction_probility = prediction_probility.withColumn('probability', probability_to_array(prediction_probility.probability))

probability_to_dict = udf(addCrimeIndex)

prediction_probility = prediction_probility.withColumn('probability', probability_to_dict(prediction_probility.probability))

pd.set_option('display.max_colwidth', None)
prediction_probility.limit(5).toPandas().head()

,Primary Type,Latitude,Longitude,District,Ward,probability
0,ASSAULT,41.852778,-87.639498,9,25,"{HOMICIDE=0.001616542656002945, BATTERY=0.20514954076131134, ROBBERY=0.046520163214909364, SEX OFFENSE=0.011366574603762255, OFFENSE INVOLVING CHILDREN=0.00817572187687132, CRIMINAL SEXUAL ASSAULT=5.995604695343382E-4, NARCOTICS=0.04361886986682161, CRIMINAL TRESPASS=0.019763611590310762, RITUALISM=2.4846815724295475E-6, CRIMINAL DAMAGE=0.13521731878695362, DECEPTIVE PRACTICE=0.04064195671205003, OTHER OFFENSE=0.0670243684427355, PUBLIC PEACE VIOLATION=0.015524781379104863, ASSAULT=0.07305309444992543, MOTOR VEHICLE THEFT=0.053729506194167394, KIDNAPPING=0.0010859134224581197, BURGLARY=0.06493450204023285, THEFT=0.211975488851276}"
1,ASSAULT,41.856510,-87.694361,10,28,"{HOMICIDE=0.0017671207071380188, BATTERY=0.20515590161714112, ROBBERY=0.046967378381913893, SEX OFFENSE=0.012582402801306, OFFENSE INVOLVING CHILDREN=0.007393911172564509, CRIMINAL SEXUAL ASSAULT=5.739181614066068E-4, NARCOTICS=0.06491769554241732, CRIMINAL TRESPASS=0.02030560184492098, RITUALISM=2.1878146115250484E-6, CRIMINAL DAMAGE=0.130785459829156, DECEPTIVE PRACTICE=0.037015727860655524, OTHER OFFENSE=0.06388518347814717, PUBLIC PEACE VIOLATION=0.017185751191400596, ASSAULT=0.07155899095237045, MOTOR VEHICLE THEFT=0.054881430277935406, KIDNAPPING=0.0010442696964975216, BURGLARY=0.058134485421132066, THEFT=0.20584258324928514}"
2,ASSAULT,41.932713,-87.649312,19,44,"{HOMICIDE=8.233940855160154E-4, BATTERY=0.16151430133712402, ROBBERY=0.038732014647190054, SEX OFFENSE=0.011117923069797996, OFFENSE INVOLVING CHILDREN=0.005303947016267672, CRIMINAL SEXUAL ASSAULT=6.839773792088695E-4, NARCOTICS=0.033191496686204476, CRIMINAL TRESPASS=0.021154751300628317, RITUALISM=1.7236541203166964E-6, CRIMINAL DAMAGE=0.12130659280790342, DECEPTIVE PRACTICE=0.06437873545475765, OTHER OFFENSE=0.06323908706589162, PUBLIC PEACE VIOLATION=0.010945810085738573, ASSAULT=0.056665073766751874, MOTOR VEHICLE THEFT=0.04368450238166964, KIDNAPPING=7.917059134983718E-4, BURGLARY=0.05747803803147431, THEFT=0.3089869253162566}"
3,ASSAULT,41.775829,-87.610698,3,20,"{HOMICIDE=0.002610361152217564, BATTERY=0.1746067716439297, ROBBERY=0.02451712805874392, SEX OFFENSE=0.03152983053162037, OFFENSE INVOLVING CHILDREN=0.006265978890055137, CRIMINAL SEXUAL ASSAULT=2.9709713836351294E-4, NARCOTICS=0.27623204351462294, CRIMINAL TRESPASS=0.057441246824446644, RITUALISM=7.185010855593436E-7, CRIMINAL DAMAGE=0.06449011101625618, DECEPTIVE PRACTICE=0.02591454259293679, OTHER OFFENSE=0.05234607871092007, PUBLIC PEACE VIOLATION=0.05247363471066082, ASSAULT=0.06159343076462707, MOTOR VEHICLE THEFT=0.02889685050511908, KIDNAPPING=6.003998149567038E-4, BURGLARY=0.02768444477736895, THEFT=0.11249933085206933}"
4,ASSAULT,41.787317,-87.744847,8,23,"{HOMICIDE=0.001715131648079039, BATTERY=0.20232855795651883, ROBBERY=0.04664528974844332, SEX OFFENSE=0.012711236004293188, OFFENSE INVOLVING CHILDREN=0.007658441595539812, CRIMINAL SEXUAL ASSAULT=5.795955035300314E-4, NARCOTICS=0.05002665682800815, CRIMINAL TRESPASS=0.020270804912569128, RITUALISM=1.9290626498617394E-6, CRIMINAL DAMAGE=0.13582015244164447, DECEPTIVE PRACTICE=0.03801394674090207, OTHER OFFENSE=0.06519354405456776, PUBLIC PEACE VIOLATION=0.01734811470265407, ASSAULT=0.07169880832292341, MOTOR VEHICLE THEFT=0.05664515676387488, KIDNAPPING=0.0010932916470713652, BURGLARY=0.06087265588854244, THEFT=0.2113766861781882}"
